In [25]:
import arviz as az
import matplotlib.pyplot as plt
import polars as ps
import pymc as pm
from theano import shared, tensor

AttributeError: partially initialized module 'theano' has no attribute 'compile' (most likely due to a circular import)

In [ ]:
az.style.use('arviz-darkgrid')

In [ ]:
dataset = ps.read_csv('dataset/howell.csv')

In [ ]:
dataset.head()

In [ ]:
over_eighteen_dataset = dataset.filter(ps.col('age') > 18)
over_eighteen_dataset_weight = dataset['weight']
over_eighteen_dataset_height = dataset['height']

In [ ]:
plt.scatter(x=over_eighteen_dataset_weight, y=over_eighteen_dataset_height)

In [ ]:
with pm.Model() as height:
	alpha = pm.Normal('α', sigma=10)
	beta = pm.Normal('β', sigma=10)
	epson = pm.HalfNormal('ϵ', sigma=10)

	weight = shared(over_eighteen_dataset_weight)
	mu = pm.Deterministic("μ", alpha + beta * weight)

	height_prediction = pm.Normal("height_pred", mu=mu, sd=epson, observed = over_eighteen_dataset_height)
	trace = pm.sample(tune=2000)
	posterior_prediction = pm.sample_posterior_predictive(trace)

In [ ]:
az.plot_trace(trace, var_names = ["α","β", "ϵ"]);